In [ ]:
import janitor
import pandas as pd

In [ ]:
RAW_DATA = "./arte_urbana_manual.xlsx"
OUTPUT_DATA = "../src/data.json"

In [ ]:
df = pd.read_excel(
    RAW_DATA,
    # index_col="ID",
    verbose=True,
    dtype={"Data": "Int64"},
)

In [ ]:
df.head()

In [ ]:
df.shape

http://geoportal.cm-amadora.pt/graffiti/Arte_Urbana/index.html

**Quotation marks**

- «
- »
- “
- ”

In [ ]:
df["Descricao_manual"].to_list()

In [ ]:
indent = 4

# df.to_json(OUTPUT_DATA, orient="records", force_ascii=False, indent=indent)

---